# 1. Combinar archivos 
#### Archivo de enrgia extraida con archivo de empresas

In [20]:
import pandas as pd
import glob
import re
import os
from pathlib import Path

# === CONFIGURACIÓN ===
CENTRALES_FILE = './data/empresas_distribuidoras.xlsx'
INPUT_PATTERN = './downloads/extracted_peaje_c_ret_*.xlsx'
OUTPUT_PREFIX = './pre_data/peaje_agentes_'

# === FUNCIONES AUXILIARES ===

def detectar_fila_encabezado(df):
    for i, row in df.iterrows():
        if any(str(cell).strip().upper() == "AGENTE" for cell in row):
            return i
    return 0

def normalizar_nombre(nombre, nombres_centrales, alias):
    x = str(nombre).strip()
    if x in alias:
        return alias[x]
    
    # Limpieza más profunda para coincidencias flexibles
    x_clean = re.sub(r'\W+', '', x).upper()
    
    for k in nombres_centrales:
        k_clean = re.sub(r'\W+', '', k).upper()
        if k_clean == x_clean:
            return k
    
    # Casos especiales para Aguaí
    if "AGUAI" in x_clean or "AGUAÍ" in x_clean:
        if "AUTOPRODUCTOR" in x_clean:
            return "Aguai (Autoproductor)"
        return "Aguaí Energia"
    
    return x

# === MAPA DE ALIAS DE CENTRALES ===
alias = {
    "Kanata en Arocagua": "Kanata ARO",
    "Kanata en Valle Hermoso": "Kanata VHE",
    "Misicuni en Arocagua": "Misicuni ARO",
    "Misicuni en Valle Hermoso": "Misicuni VHE",
    "Yunchara": "Yunchara",
    "Aguaí Energía": "Aguaí Energia",
    "AGUAÍ ENERGÍA S.A.": "Aguaí Energia",
    "Santa Cruz (Aguaí)": "Santa Cruz (Aguaí)",
    "RÍO ELÉCTRICO S.A.": "RIO ELECTRICO S.A.",
    "CHACO ENERGÍAS S.A.": "CHACO ENERGIAS S.A.",
    "RIOELEC S.A.": "RIO ELECTRICO S.A.",
}

# === MAPA EXHAUSTIVO DE AGENTES A EMPRESAS ===
mapeo_agente_empresa = {
    # CRE
    "Guaracachi": "CRE",
    "Urubó": "CRE",
    "Urubó 115 kV": "CRE",
    "Urubó 115": "CRE",
    "Arboleda": "CRE",
    "Warnes": "CRE",
    "Brechas": "CRE",
    "Brechas 69": "CRE",
    "Brechas 115": "CRE",
    "Troncos": "CRE",
    "Troncos 115": "CRE",
    "Troncos - Las Misiones": "CRE",
    "Troncos 115 (Las Misiones)": "CRE",
    "Yapacaní": "CRE",
    "Bélgica": "CRE",
    "San Julián": "CRE",
    "Camiri - Cordillera": "CRE",
    "Guarayos": "CRE",
    
    # DELAPAZ
    "Kenko": "DELAPAZ",
    "Cumbre": "DELAPAZ",
    "Chuspipata": "DELAPAZ",
    "Caranavi": "DELAPAZ",
    "San Buenaventura": "DELAPAZ",
    "Palca": "DELAPAZ",
    "Contorno Bajo": "DELAPAZ",
    "Choquetanga": "DELAPAZ",
    
    # ELFEC
    "Arocagua": "ELFEC",
    "Valle Hermoso": "ELFEC",
    "Irpa Irpa": "ELFEC",
    "Chimore": "ELFEC",
    "San José": "ELFEC",
    "Paracaya": "ELFEC",
    "Carrasco": "ELFEC",
    "Qollpana": "ELFEC",
    "Villa Tunari": "ELFEC",
    "Santivañez": "ELFEC",
    
    # ENDE DEORURO
    "Vinto": "ENDE DEORURO S.A.",
    "Vinto 115": "ENDE DEORURO S.A.",
    "Catavi": "ENDE DEORURO S.A.",
    "Jeruyo": "ENDE DEORURO S.A.",
    "Lucianita": "ENDE DEORURO S.A.",
    "Catavi 115": "ENDE DEORURO S.A.",
    "Pagador": "ENDE DEORURO S.A.",
    
    # SEPSA
    "Sacaca": "SEPSA",
    "Ocuri": "SEPSA",
    "Potosí": "SEPSA",
    "Potosí 115": "SEPSA",
    "Potosi 69": "SEPSA",
    "Potosi 115": "SEPSA",
    "Punutuma": "SEPSA",
    "Don Diego": "SEPSA",
    "CM  Karachipampa": "SEPSA",
    "Litio - Lipez": "SEPSA",
    "Litio 115 kV": "SEPSA",
    "Torre Huayco": "SEPSA",
    "Portugalete": "SEPSA",
    "Chilcobija": "SEPSA",
    "Telamayu": "SEPSA",
    "La Plata": "SEPSA",
    "ECEBOL Potosí": "SEPSA",
    
    # CESSA
    "Mariaca": "CESSA",
    "Sucre": "CESSA",
    "Sucre - Fancesa": "CESSA",
    "Sucre 115": "CESSA",
    
    # ENDE
    "Tazna": "ENDE",
    "Uyuni": "ENDE",
    "Uyuni - Uyuni": "ENDE",
    "Las Carreras": "ENDE",
    
    # SETAR
    "Tarija": "SETAR",
    "Villamontes": "SETAR",
    "Yacuiba": "SETAR",
    "Bermejo": "SETAR",
    
    # ENDE DELBENI
    "Yucumo": "ENDE DELBENI S.A.M.",
    "San Borja": "ENDE DELBENI S.A.M.",
    "San Ignacio de Moxos": "ENDE DELBENI S.A.M.",
    "Trinidad": "ENDE DELBENI S.A.M.",
    "Paraíso": "ENDE DELBENI S.A.M.",
    
    # NO REGULADOS
    "EMDEECRUZ": "NO REGULADOS",
    "EMVINTO - COMIBOL": "NO REGULADOS",
    "COBOCE": "NO REGULADOS",
    "MINERA SAN CRISTOBAL S.A.": "NO REGULADOS",
    "YLB (Contrato ENDE)": "NO REGULADOS",
    "LAS LOMAS": "NO REGULADOS",
    "CERAMICA GUADALQUIVIR": "NO REGULADOS",
    "EMPACAR S.A.": "NO REGULADOS",
    
    # AGUAÍ
    "Aguaí Energia": "AGUAÍ ENERGÍA S.A.",
    "Aguai (Autoproductor)": "AGUAÍ ENERGÍA S.A."
}

# === PROCESAMIENTO DE ARCHIVOS ===

def procesar_archivos():
    try:
        df_centrales = pd.read_excel(CENTRALES_FILE)
        df_centrales['AGENTE'] = df_centrales['AGENTE'].astype(str).str.strip()

        if not {'AGENTE', "EMPRESA"}.issubset(df_centrales.columns):
            print("Error: El archivo debe contener columnas 'AGENTE', y 'EMPRESA'")
            return

        mapeo_generadores = dict(zip(df_centrales['AGENTE'], df_centrales['EMPRESA']))
        nombres_centrales = set(df_centrales['AGENTE'])

        # Combinar mapeos
        mapeo_completo = {**mapeo_generadores, **mapeo_agente_empresa}

    except Exception as e:
        print(f"Error al cargar archivo de mapeo {CENTRALES_FILE}: {str(e)}")
        return

    for input_file in glob.glob(INPUT_PATTERN):
        file_number = re.search(r'extracted_peaje_c_ret_(\d+)\.xlsx', input_file)

        if not file_number:
            continue

        file_number = file_number.group(1)
        output_file = f"{OUTPUT_PREFIX}{file_number}.xlsx"

        if os.path.exists(output_file):
            print(f"[Omitido] {output_file} ya existe.")
            continue

        try:
            df_raw = pd.read_excel(input_file, header=None)
            start_row = detectar_fila_encabezado(df_raw)
            df = pd.read_excel(input_file, skiprows=start_row)
            df.columns = [str(col).strip() for col in df.columns]
            
            # Manejar columnas duplicadas
            if df.columns.duplicated().any():
                df = df.loc[:, ~df.columns.duplicated(keep='first')]

            central_col = next((c for c in df.columns if 'central' in c.lower() or 'agente' in c.lower()), None)
            if central_col and central_col != 'AGENTE':
                df = df.rename(columns={central_col: 'AGENTE'})
            if 'AGENTE' not in df.columns:
                print(f"[Error] No se encontró columna 'AGENTE' en {input_file}")
                continue

            df['AGENTE'] = df['AGENTE'].astype(str).str.strip()

            # Eliminación de filas basura
            pattern_basura = r'TOTAL|TOTALES|Nota|Tipo de cambio|nan|CARGOS POR INYECCIONES|TOTAL\s*-\s*AGUAI'
            df = df[~df['AGENTE'].str.contains(pattern_basura, case=False, na=True, regex=True)]
            df = df[~df['AGENTE'].str.match(r'^\d{4}-\d{2}-\d{2}', na=False)]
            df = df[~df['AGENTE'].str.upper().str.contains(r'AGENTE\s*ENERGIA|POTENCIA', na=False)]
            df = df[df['AGENTE'].notna() & (df['AGENTE'] != '')]

            # Identificación de filas válidas
            df['AGENTE_CLEAN'] = df['AGENTE'].str.strip().str.upper()
            centrales_validas = set(x.upper() for x in nombres_centrales)
            first_valid_idx = df[df['AGENTE_CLEAN'].isin(centrales_validas)].index.min()
            if pd.isna(first_valid_idx):
                print(f"[Error] No se encontraron centrales válidas en {input_file}")
                continue
            df = df.loc[first_valid_idx:].copy()

            # Normalización
            df['AGENTE_NORMALIZADA'] = df['AGENTE'].apply(
                lambda x: normalizar_nombre(x, nombres_centrales, alias)
            )
            
            # CORRECCIÓN PRINCIPAL: Asignación completa de empresas
            df['EMPRESA'] = df['AGENTE_NORMALIZADA'].map(mapeo_completo)
            
            # Forzar presencia de ambas centrales Aguaí
            aguai_centrales = ["Aguaí Energia", "Aguai (Autoproductor)"]
            for central in aguai_centrales:
                if central not in df['AGENTE_NORMALIZADA'].values:
                    nueva_fila = {'AGENTE_NORMALIZADA': central, 'EMPRESA': "AGUAÍ ENERGÍA S.A."}
                    for col in df.columns:
                        if 'kW' in col or 'kWh' in col or 'USD' in col:
                            nueva_fila[col] = 0.0
                    df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)

            # Procesamiento numérico
            for col in df.columns:
                if 'kW' in col or 'kWh' in col or 'USD' in col:
                    df[col] = (
                        df[col]
                        .astype(str)
                        .str.replace(',', '')
                        .str.replace(' ', '')
                        .replace('nan', None)
                        .astype(float)
                    )

            # Manejo de columnas de peaje
            peaje_cols = [col for col in df.columns if 'Peaje' in col and 'USD' in col]
            columnas_conservar = ['AGENTE_NORMALIZADA', 'EMPRESA'] + peaje_cols
            df_final = df[columnas_conservar].copy()
            df_final = df_final.rename(columns={'AGENTE_NORMALIZADA': 'AGENTE'})

            # Renombrar columnas específicas
            rename_columns = {
                'Energía': 'Energía kWh',
                'Potencia Firme Remunerada': 'Potencia kW'
            }
            df_final = df_final.rename(columns=rename_columns)

            # Guardar resultados
            df_final.to_excel(output_file, index=False)
            print(f"[OK] {input_file} → {output_file}")

            # Detectar centrales sin mapeo
            faltantes = df_final[df_final['EMPRESA'].isna()]['AGENTE'].unique()
            if len(faltantes) > 0:
                print(f"  ⚠️ {len(faltantes)} centrales sin EMPRESA: {faltantes[:3]}{'...' if len(faltantes) > 3 else ''}")

        except Exception as e:
            print(f"[Error] {input_file}: {str(e)}")
            try:
                Path('./errors').mkdir(exist_ok=True)
                df.to_excel(f"./errors/ERROR_{file_number}.xlsx", index=False)
            except Exception as inner_e:
                print(f"Error al guardar archivo de error: {inner_e}")

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    print("🔄 Iniciando procesamiento de archivos...")
    procesar_archivos()
    print("✅ Proceso completado.")

🔄 Iniciando procesamiento de archivos...
[OK] ./downloads\extracted_peaje_c_ret_0123.xlsx → ./pre_data/peaje_agentes_0123.xlsx
[OK] ./downloads\extracted_peaje_c_ret_0124.xlsx → ./pre_data/peaje_agentes_0124.xlsx
  ⚠️ 3 centrales sin EMPRESA: ['Yapacani' 'Chimoré' 'Sucre - Aranjuez']
[OK] ./downloads\extracted_peaje_c_ret_0125.xlsx → ./pre_data/peaje_agentes_0125.xlsx
[OK] ./downloads\extracted_peaje_c_ret_0223.xlsx → ./pre_data/peaje_agentes_0223.xlsx
[OK] ./downloads\extracted_peaje_c_ret_0224.xlsx → ./pre_data/peaje_agentes_0224.xlsx
  ⚠️ 3 centrales sin EMPRESA: ['Yapacani' 'Chimoré' 'Sucre - Aranjuez']
[OK] ./downloads\extracted_peaje_c_ret_0225.xlsx → ./pre_data/peaje_agentes_0225.xlsx
[OK] ./downloads\extracted_peaje_c_ret_0323.xlsx → ./pre_data/peaje_agentes_0323.xlsx
[OK] ./downloads\extracted_peaje_c_ret_0324.xlsx → ./pre_data/peaje_agentes_0324.xlsx
[OK] ./downloads\extracted_peaje_c_ret_0325.xlsx → ./pre_data/peaje_agentes_0325.xlsx
[OK] ./downloads\extracted_peaje_c_ret_04

# 2. Merge archivos energia

In [24]:
import pandas as pd
import glob
from datetime import datetime

def consolidar_en_formato_largo():
    archivos = sorted(glob.glob('./pre_data/peaje_agentes_*.xlsx'))
    if not archivos:
        print("No se encontraron archivos.")
        return

    registros = []

    for archivo in archivos:
        try:
            # Extraer mes y año del nombre del archivo
            periodo = archivo.split('_')[-1].split('.')[0]
            mes = int(periodo[:2])
            año = 2000 + int(periodo[2:])
            fecha = datetime(año, mes, 1)

            df = pd.read_excel(archivo)

            # Verificar columnas esenciales
            if 'AGENTE' not in df.columns:
                print(f"Omitido: {archivo} no tiene columna AGENTE.")
                continue

            columnas_necesarias = ['AGENTE', 'EMPRESA']
            columnas_datos = [col for col in df.columns if col not in columnas_necesarias]
            
            # Si no hay columnas de datos, omitir
            if not columnas_datos:
                print(f"Omitido: {archivo} no tiene columnas de datos.")
                continue

            # Si no existe columna TECNOLOGIA, crear una con NaN
            if 'TECNOLOGIA' not in df.columns:
                df['TECNOLOGIA'] = None

            # Derretir (melt) el DataFrame para convertir a formato largo
            temp = pd.melt(
                df,
                id_vars=['AGENTE', 'EMPRESA'],
                value_vars=columnas_datos,
                var_name='VARIABLE',
                value_name='VALOR'
            )
            
            temp['FECHA'] = fecha
            registros.append(temp)

        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
            continue

    if not registros:
        print("No se pudo consolidar ningún archivo válido.")
        return

    df_largo = pd.concat(registros, ignore_index=True)
    df_largo = df_largo[['FECHA', "AGENTE" , 'EMPRESA', 'VARIABLE', 'VALOR']]

    df_largo.to_excel("./preprocess/serie_peaje_filiales.xlsx", index=False)
    print("Consolidación completada en formato largo.")
    print(f"Filas totales: {len(df_largo)}")
    print("Archivo guardado como 'serie_peaje_filiales.xlsx'")

if __name__ == "__main__":
    consolidar_en_formato_largo()


Consolidación completada en formato largo.
Filas totales: 9316
Archivo guardado como 'serie_peaje_filiales.xlsx'


# 3. Creacion de serie de tiempo de energia

In [25]:
import pandas as pd

# Cargar archivo en formato largo
df = pd.read_excel("./preprocess/serie_peaje_filiales.xlsx")

# Asegurar que FECHA sea tipo datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Formatear fecha como MMYYYY (ejemplo: 012023)
df['PERIODO'] = df['FECHA'].dt.strftime("%m%Y")

# Crear columna combinada: VARIABLE + PERIODO
df['COLUMNA'] = df['VARIABLE'] + ' ' + df['PERIODO']

# Pivotear: una fila por CENTRAL, GENERADOR, TECNOLOGÍA
tabla_pivot = df.pivot_table(
    index=['AGENTE', 'EMPRESA'],
    columns='COLUMNA',
    values='VALOR',
    aggfunc='sum'  # En caso de duplicados, los suma
)

# Resetear índice para que sea un DataFrame plano
tabla_pivot = tabla_pivot.reset_index()

# --- NUEVO CÓDIGO PARA ORDENAR COLUMNAS CRONOLÓGICAMENTE ---
# 1. Separar las columnas en componentes
fixed_cols = ['AGENTE', 'EMPRESA']
other_cols = [c for c in tabla_pivot.columns if c not in fixed_cols]

# 2. Crear DataFrame temporal para ordenación
cols_df = pd.DataFrame({
    'col_name': other_cols,
    'variable': [c.split()[0] for c in other_cols],  # Ej: 'Energía MWh'
    'periodo': [c.split()[-1] for c in other_cols]   # Ej: '012023'
})

# 3. Convertir periodo a fecha para ordenar correctamente
cols_df['fecha'] = pd.to_datetime(cols_df['periodo'], format='%m%Y')

# 4. Ordenar por: 1. Tipo de variable, 2. Fecha cronológica
cols_df = cols_df.sort_values(by=['variable', 'fecha'])

# 5. Construir lista final de columnas ordenadas
ordered_columns = fixed_cols + cols_df['col_name'].tolist()

# 6. Aplicar nuevo orden al DataFrame
tabla_pivot = tabla_pivot[ordered_columns]
# --- FIN DEL NUEVO CÓDIGO ---

# Guardar resultado
tabla_pivot.to_excel("./preprocess/serie_peaje_filiales_2.xlsx", index=False)
print("✅ Archivo guardado como 'serie_peaje_filiales_2.xlsx'")





✅ Archivo guardado como 'serie_peaje_filiales_2.xlsx'


# 4. Depuracion de serie de tiempo de peaje

In [26]:
import pandas as pd

# Cargar archivo original
df = pd.read_excel("./preprocess/serie_peaje_filiales_2.xlsx")

# Columnas fijas
fixed_cols = ['AGENTE', 'EMPRESA']

# Identificar columnas de peaje por mes
peaje_cols = [col for col in df.columns if 'Peaje' in col and any(x in col for x in ['ENDE Trans.', 'ENDE USD', 'ISA', 'TESA', 'filiales'])]

# Obtener lista de fechas únicas (ej. 012023, 022023, etc.)
fechas = sorted(set(col.split()[-1] for col in peaje_cols))

# Crear diccionario con sumas por mes
peaje_generacion = pd.DataFrame(df[fixed_cols])

for fecha in fechas:
    columnas_mes = [col for col in peaje_cols if col.endswith(fecha)]
    peaje_generacion[f'Peaje generación USD/MWh {fecha}'] = df[columnas_mes].sum(axis=1)

# Ordenar columnas cronológicamente (después de las fijas)
cols_ordenadas = fixed_cols + sorted([col for col in peaje_generacion.columns if col.startswith('Peaje generación')], key=lambda x: pd.to_datetime(x.split()[-1], format='%m%Y'))
peaje_generacion = peaje_generacion[cols_ordenadas]

# Guardar archivo final
peaje_generacion.to_excel("./data/serie_peaje.xlsx", index=False)
print("✅ Archivo generado con columna 'Peaje generación USD/MWh' en orden cronológico.")



✅ Archivo generado con columna 'Peaje generación USD/MWh' en orden cronológico.
